<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Первая-область" data-toc-modified-id="Первая-область-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Первая область</a></span></li><li><span><a href="#Вторая-область" data-toc-modified-id="Вторая-область-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Вторая область</a></span></li><li><span><a href="#Третья-область" data-toc-modified-id="Третья-область-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Третья область</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Обучение-и-проверка-модели" data-toc-modified-id="Обучение-и-проверка-модели-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение и проверка модели</a></span><ul class="toc-item"><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обучение моделей</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Подготовка-к-расчёту-прибыли" data-toc-modified-id="Подготовка-к-расчёту-прибыли-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Подготовка к расчёту прибыли</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Расчёт-прибыли-и-рисков" data-toc-modified-id="Расчёт-прибыли-и-рисков-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Расчёт прибыли и рисков</a></span><ul class="toc-item"><li><span><a href="#Прибыль" data-toc-modified-id="Прибыль-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Прибыль</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Риски" data-toc-modified-id="Риски-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Риски</a></span></li></ul></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# Условия задачи

1. Для обучения модели подходит **только линейная регрессия** (остальные — недостаточно предсказуемые).


2. При разведке региона **исследуют 500 точек**, из которых с помощью машинного обучения **выбирают 200 лучших** для разработки.


3. Бюджет на разработку скважин в регионе — **10 млрд рублей.**


4. При нынешних ценах **один баррель сырья** приносит **450 рублей дохода**. Доход с каждой единицы продукта составляет **450 тыс. рублей**, поскольку объём указан в тысячах баррелей.


5. После оценки рисков нужно оставить лишь те регионы, в которых **вероятность убытков меньше 2.5%.** Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка и подготовка данных

Производим **импорт библиотек.**

In [25]:
import pandas as pd 

In [2]:
import numpy as np
from numpy.random import RandomState

In [3]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, mean_squared_error

**Добавляем глобальные константы.**

In [4]:
WELL_DEVELOPMENT = 10_000_000_000
PRICE = 450_000

**Записываем данные** из csv-файла в переменные.

In [27]:
key = '%D0%9C%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B5%20%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5%20%D0%B2%20%D0%B1%D0%B8%D0%B7%D0%BD%D0%B5%D1%81%D0%B5'
list_ = []

for i in range(0, 3):
    url = f'https://github.com/MEfse/project/blob/main/Project/{key}/geo_data_{i}.csv?raw=true'
    list_.append(url)

In [28]:
geo_data_0 = pd.read_csv(list_[0])
geo_data_1 = pd.read_csv(list_[1])
geo_data_2 = pd.read_csv(list_[2])

Создаем функцию по **отображению данных.**

In [6]:
def info(data):
    data.info()
    display(data.head(5))
    display(data.tail(5))

### Первая область

In [7]:
info(geo_data_0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764
99999,1CWhH,1.764754,-0.266417,5.722849,149.633246


### Вторая область

In [8]:
info(geo_data_1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364
99999,relB0,-3.426139,-7.794274,-0.003299,3.179103


### Третья область

In [9]:
info(geo_data_2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,id,f0,f1,f2,product
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253
99999,V9kWn,-2.551421,-2.025625,6.090891,102.775767


### Вывод

**Удалим  неинформативный  столбец** для моделей машинного обучения.

In [10]:
geo_data_0 = geo_data_0.drop(['id'], axis=1)
geo_data_1 = geo_data_1.drop(['id'], axis=1)
geo_data_2 = geo_data_2.drop(['id'], axis=1)

**Значение столбцов.**

* `f0`, `f1`, `f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` — объём запасов в скважине (тыс. баррелей).

**Проверка на дубликаты.**

In [11]:
def duplicat(data):
    return data.duplicated().sum()

In [12]:
count_duplicat = pd.DataFrame({'Количество дубликатов в первой области' : [duplicat(geo_data_0)],
                              'Количество дубликатов в второй области' : [duplicat(geo_data_1)],
                              'Количество дубликатов в третьей области' : [duplicat(geo_data_2)]}, index=['count']).T

In [13]:
display(count_duplicat)

,count
Количество дубликатов в первой области,0
Количество дубликатов в второй области,0
Количество дубликатов в третьей области,0


**Пропущенных значений и дубликатов не наблюдается. Данные в нормальном состоянии**, поэтому приступаем к обучению и последующей проверке модели.

## Обучение и проверка модели

### Обучение моделей

**Производим деление отобранных данных** на обучающую и валидационную выборки.

In [14]:
def data_division(data):
    features_geo = data.drop(['product'], axis=1)
    target_geo = data['product']

    X_train, X_valid, y_train, y_valid = train_test_split(features_geo, target_geo, test_size=0.25, random_state=12345)
    
    return X_train, X_valid, y_train, y_valid  

In [15]:
X_train_0, X_valid_0, y_train_0, y_valid_0 = data_division(geo_data_0)
X_train_1, X_valid_1, y_train_1, y_valid_1 = data_division(geo_data_1)
X_train_2, X_valid_2, y_train_2, y_valid_2 = data_division(geo_data_2)

**Выведим размер выборок.**

In [16]:
print('Размер обучающей выборки', X_train_0.shape)
print('Размер валидационной выборки', X_valid_1.shape)

Размер обучающей выборки (75000, 3)
Размер валидационной выборки (25000, 3)


In [19]:
def predict(y_valid, predict):
    
    y_valid = pd.Series(y_valid)
    predict = pd.Series(predict)

    data_y_valid = y_valid.to_frame().reset_index(drop=True)
    data_predict = predict.to_frame().reset_index(drop=True)
    
    data_y_valid = data_y_valid.join(data_predict)
    data_y_valid.columns = ['y_valid', 'predict'] 

    data_y_valid['difference'] = abs(data_y_valid['y_valid'] - data_y_valid['predict'])
    data_y_valid = data_y_valid.sort_values('predict', ascending=False)
    
    predict_mean = round(data_y_valid['predict'].mean(), 2)
    
    mse = mean_squared_error(data_y_valid['y_valid'], data_y_valid['predict']).round(2)
    rmse = (mse ** 0.5).round(2)
    
    display(data_y_valid.head(5))
    display(data_y_valid.tail(5))
    
    return rmse, predict_mean

**Первая область**

In [20]:
model_0 = LinearRegression()
model_0.fit(X_train_0, y_train_0)

predict_0 = model_0.predict(X_valid_0)

rmse_0, predict_mean_0 = predict(y_valid_0, predict_0)

print("RMSE =", rmse_0)
print("Cредний запас предсказанного сырья =", predict_mean_0)

,y_valid,predict,difference
9317,162.810993,180.180713,17.369720
219,153.639837,176.252213,22.612376
10015,162.153488,175.850623,13.697135
11584,96.893581,175.658429,78.764848
23388,178.879516,173.299686,5.579830


,y_valid,predict,difference
23129,49.374929,10.209351,39.165578
9078,25.956984,8.798531,17.158453
21875,46.546824,6.569477,39.977347
14503,11.018692,4.980987,6.037705
20881,25.069622,-9.367846,34.437468


RMSE = 37.58
Cредний запас предсказанного сырья = 92.59


**Вторая область**

In [21]:
model_1 = LinearRegression()
model_1.fit(X_train_1, y_train_1)

predict_1 = model_1.predict(X_valid_1)

rmse_1, predict_mean_1 = predict(y_valid_1, predict_1)

print("RMSE =", rmse_1)
print("Cредний запас предсказанного сырья =", predict_mean_1)

,y_valid,predict,difference
20430,137.945408,139.818970,1.873562
7777,137.945408,139.773423,1.828015
8755,137.945408,139.703330,1.757923
1178,137.945408,139.560938,1.615530
4285,137.945408,139.516754,1.571347


,y_valid,predict,difference
5085,0.0,-1.634717,1.634717
12426,0.0,-1.693844,1.693844
9852,0.0,-1.871442,1.871442
5617,0.0,-1.883737,1.883737
4975,0.0,-1.893774,1.893774


RMSE = 0.89
Cредний запас предсказанного сырья = 68.73


**Третья область**

In [22]:
model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2)

predict_2 = model_2.predict(X_valid_2)

rmse_2, predict_mean_2 = predict(y_valid_2, predict_2)

print("RMSE =", rmse_2)
print("Cредний запас предсказанного сырья =", predict_mean_2)

,y_valid,predict,difference
22636,175.103291,165.856833,9.246458
24690,131.627481,165.679685,34.052204
7811,141.160070,163.439962,22.279893
1581,159.676082,162.062589,2.386507
6751,142.135203,161.797476,19.662273


,y_valid,predict,difference
21032,33.411204,23.323600,10.087604
18206,60.244891,20.393911,39.850980
11296,62.903882,20.189540,42.714342
5298,57.022698,17.297411,39.725287
22577,32.496867,17.131597,15.365270


RMSE = 40.03
Cредний запас предсказанного сырья = 94.97


### Вывод

In [23]:
data_model = (pd
              .DataFrame({'Скважины' : ['Первая','Вторая','Третья'],
                          'RMSE' : [rmse_0, rmse_1, rmse_2],
                          'Cр. запас пред. сырья' : [predict_mean_0, predict_mean_1, predict_mean_2]})
              .sort_values('Cр. запас пред. сырья', ascending=False))

In [24]:
display(data_model)

,Скважины,RMSE,Cр. запас пред. сырья
2,Третья,40.03,94.97
0,Первая,37.58,92.59
1,Вторая,0.89,68.73


Заметим, что средний объем предсказанного сырья **практически равен в первой и третьей областях**, при этом **во второй сырья меньше примерно на 30%**, так же обратим внимание на параметр RMSE, и чем он ниже тем лучше. **Во второй области наблюдаем невероятно низкое значение RMSE (по сравнению с другими областями)**, что говорит о маленьком разбросе в данных. Более детальный анализ (расчёт прибыли) выявит наиболее подходящие (прибыльные) области.

## Подготовка к расчёту прибыли

Отобразим переменные описанные выше.

***
* `rmse_0` - RMSE первой области.
* `rmse_1` - RMSE второй области.
* `rmse_2` - RMSE третьей области.
***
* `predict_mean_0` - Cредний запас предсказанного сырья первой области.
* `predict_mean_1` - Cредний запас предсказанного сырья второй области.
* `predict_mean_2` - Cредний запас предсказанного сырья третьей области.
***

**Достаточный объём**

In [34]:
#стоимость_разработки_одной_скважины
single_point_development = WELL_DEVELOPMENT / 200 

#стоимость_разработки_одной_скважины / цена_одной_единицы_сырья
volume = (single_point_development / PRICE) 
volume = round(volume, 2)
print(f'Объем для безубыточной разработки: \033[1m{volume}')

Объем для безубыточной разработки: 111.11


### Вывод

In [35]:
data_volume = (pd
              .DataFrame({'Скважины' : ['Первая','Вторая','Третья'],
                          'Необход. объем' : [volume, volume, volume],
                          'Cр. запас пред. сырья' : [predict_mean_0, predict_mean_1, predict_mean_2]})
              .sort_values('Cр. запас пред. сырья', ascending=False))

display(data_volume)

,Скважины,Необход. объем,Cр. запас пред. сырья
2,Третья,111.11,94.97
0,Первая,111.11,92.59
1,Вторая,111.11,68.73


Области не имеют **достаточного минимального объема для получения дохода.** Области имеют недостаточно сырья в среднем, если брать случайные точки для разработки. Это подводит нас к целесообразности использования процедуры bootstrap, потому что без МО разработка будет, скорее всего, убыточной.

## Расчёт прибыли и рисков 

### Прибыль

Создаем функцию для получения прибыли. На вход подается цель, предсказания, количество скважин, и константы. На выход функции идет прибыль.

In [36]:
def revenue(target, predict, count, WELL_DEVELOPMENT, PRICE):
    
    target_s = pd.Series(target).reset_index(drop=True)
    predict_s = pd.Series(predict).reset_index(drop=True)
    
    selection = predict_s.sort_values(ascending=False).head(200)
    target_part = target_s[selection.index]
    
    income = int(target_part.sum() * PRICE - WELL_DEVELOPMENT)
    
    return income

**Прибыль первой области**

In [40]:
income_0 = revenue(y_valid_0, predict_0, 500, WELL_DEVELOPMENT, PRICE)
print(f'Прибыль первой области равна \033[1m{income_0}')

Прибыль первой области равна 3320826043


**Прибыль второй области**

In [41]:
income_1 = revenue(y_valid_1, predict_1, 500, WELL_DEVELOPMENT, PRICE)
print(f'Прибыль второй области равна \033[1m{income_1}')

Прибыль второй области равна 2415086696


**Прибыль третьей области**

In [42]:
income_2 = revenue(y_valid_2, predict_2, 500, WELL_DEVELOPMENT, PRICE)
print(f'Прибыль третьей области равна \033[1m{income_2}')

Прибыль третьей области равна 2710349963


### Вывод

In [43]:
data_income = (pd
              .DataFrame({'Скважины' : ['Первая','Вторая','Третья'],
                          'RMSE' : [rmse_0, rmse_1, rmse_2],
                          'Cр. запас пред. сырья' : [predict_mean_0, predict_mean_1, predict_mean_2],
                          'Прибыль' : [income_0, income_1, income_2],})
              .sort_values('Прибыль', ascending=False))

display(data_income)

,Скважины,RMSE,Cр. запас пред. сырья,Прибыль
0,Первая,37.58,92.59,3320826043
2,Третья,40.03,94.97,2710349963
1,Вторая,0.89,68.73,2415086696


### Риски

In [44]:
def bootstrap(target, predict, count, WELL_DEVELOPMENT, PRICE):
    
    values = []
    count = 0
    
    target_s = pd.Series(target).reset_index(drop=True)
    predict_s = pd.Series(predict).reset_index(drop=True)
    
    for i in range(1000):
    
        selection = predict_s.sample(n=500, replace=True, random_state=state)
        target_part = target_s[selection.index]
        
        income = revenue(target_part, selection, count, WELL_DEVELOPMENT, PRICE)
        
        if income < 0:
            count += 1
        
        values.append(income)
        
        

    values = pd.Series(values)
    mean = values.mean()
    interval_l = values.quantile(0.025)
    risk = (count / len(values) * 100)
    interval_u = values.quantile(0.975)
    

    return mean, interval_l, interval_u, risk

In [45]:
state = np.random.RandomState(12345)

In [46]:
def risk(y_valid, predict):
    mean = bootstrap(y_valid, predict, 500, WELL_DEVELOPMENT, PRICE)[0]
    print('Cредняя прибыль:', mean)
    in_l = bootstrap(y_valid, predict, 500, WELL_DEVELOPMENT, PRICE)[1]
    print('Нижняя допустимая планка по рискам:', in_l)
    in_u = bootstrap(y_valid, predict, 500, WELL_DEVELOPMENT, PRICE)[2]
    print(f'95%-й доверительный интервал: ( {in_l} - {in_u} )')
    risk = bootstrap(y_valid, predict, 500, WELL_DEVELOPMENT, PRICE)[3]
    print('Риск убытков:', risk, '%')
    
    return mean, in_l, in_u, risk 

**Риски первой области**

In [47]:
risk(y_valid_0, predict_0)

Cредняя прибыль: 396164984.366
Нижняя допустимая планка по рискам: -142969647.9
95%-й доверительный интервал: ( -142969647.9 - 954523566.8249998 )
Риск убытков: 5.7 %


(396164984.366, -142969647.9, 954523566.8249998, 5.7)

**Риски второй области**

In [48]:
risk(y_valid_1, predict_1)

Cредняя прибыль: 448275288.747
Нижняя допустимая планка по рискам: 54478628.32500001
95%-й доверительный интервал: ( 54478628.32500001 - 829550749.6499999 )
Риск убытков: 1.4000000000000001 %


(448275288.747, 54478628.32500001, 829550749.6499999, 1.4000000000000001)

**Риски третьей области**

In [49]:
risk(y_valid_2, predict_2)

Cредняя прибыль: 385987233.95
Нижняя допустимая планка по рискам: -149174500.9
95%-й доверительный интервал: ( -149174500.9 - 905407991.7499999 )
Риск убытков: 7.000000000000001 %


(385987233.95, -149174500.9, 905407991.7499999, 7.000000000000001)

## Общий вывод

In [50]:
conclusion = (pd
              .DataFrame({'Скважины' : ['Первая','Вторая','Третья'],
                          'Cредняя прибыль, руб' : [int(bootstrap(y_valid_0, predict_0, 200, WELL_DEVELOPMENT, PRICE)[0]),
                                               bootstrap(y_valid_1, predict_1, 200, WELL_DEVELOPMENT, PRICE)[0], 
                                               bootstrap(y_valid_2, predict_2, 200, WELL_DEVELOPMENT, PRICE)[0]],
                          'Нижняя граница интер.' : [bootstrap(y_valid_0, predict_0, 200, WELL_DEVELOPMENT, PRICE)[1],
                                               bootstrap(y_valid_1, predict_1, 200, WELL_DEVELOPMENT, PRICE)[1], 
                                               bootstrap(y_valid_2, predict_2, 200, WELL_DEVELOPMENT, PRICE)[1]],
                          'Верхняя граница интер.' : [bootstrap(y_valid_0, predict_0, 200, WELL_DEVELOPMENT, PRICE)[2],
                                               bootstrap(y_valid_1, predict_1, 200, WELL_DEVELOPMENT, PRICE)[2], 
                                               bootstrap(y_valid_2, predict_2, 200, WELL_DEVELOPMENT, PRICE)[2]],
                          'Риск убытков, %' : [bootstrap(y_valid_0, predict_0, 200, WELL_DEVELOPMENT, PRICE)[3],
                                               bootstrap(y_valid_1, predict_1, 200, WELL_DEVELOPMENT, PRICE)[3], 
                                               bootstrap(y_valid_2, predict_2, 200, WELL_DEVELOPMENT, PRICE)[3]]})
              .sort_values('Cредняя прибыль, руб', ascending=False))

display(conclusion)

,Скважины,"Cредняя прибыль, руб",Нижняя граница интер.,Верхняя граница интер.,"Риск убытков, %"
1,Вторая,4.561254e+08,5.511773e+07,8.765660e+08,2.2
0,Первая,3.878500e+08,-1.339742e+08,8.638442e+08,6.5
2,Третья,3.842937e+08,-1.662260e+08,8.989831e+08,7.7


В данной работе собраны характеристики для скважин: 
* качество нефти и объём её запасов; 
* построена модель для предсказания объёма запасов в новых скважинах; 
* выбирана скважины с самыми высокими оценками значений.


**Регион с максимальной суммарной прибылью отобранных скважин - второй.** Добывающей компании «ГлавРосГосНефть» не стоит бояться вкладывать деньги, т.к. **риск убытков 2.2%**, а средняя прибыль входит в доверительный интервал и относительно далек до нижней границы.


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован